In [3]:
import json
import numpy as np
import os
import cv2
import scipy.io as sio
import copy
import random
from glob import glob
from tqdm import tqdm
import sys
from pathlib import Path
%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [3]:
def load_BFM(model_path):
    ''' load BFM 3DMM model
    Args:
        model_path: path to BFM model. 
    Returns:
        model: (nver = 53215, ntri = 105840). nver: number of vertices. ntri: number of triangles.
            'shapeMU': [3*nver, 1]
            'shapePC': [3*nver, 199]
            'shapeEV': [199, 1]
            'expMU': [3*nver, 1]
            'expPC': [3*nver, 29]
            'expEV': [29, 1]
            'texMU': [3*nver, 1]
            'texPC': [3*nver, 199]
            'texEV': [199, 1]
            'tri': [ntri, 3] (start from 1, should sub 1 in python and c++)
            'tri_mouth': [114, 3] (start from 1, as a supplement to mouth triangles)
            'kpt_ind': [68,] (start from 1)
    PS:
        You can change codes according to your own saved data.
        Just make sure the model has corresponding attributes.
    '''
    C = sio.loadmat(model_path)
    model = C['model']
    model = model[0, 0]

    # change dtype from double(np.float64) to np.float32,
    # since big matrix process(espetially matrix dot) is too slow in python.
    model['shapeMU'] = (model['shapeMU'] + model['expMU']).astype(np.float32)
    model['shapePC'] = model['shapePC'].astype(np.float32)
    model['shapeEV'] = model['shapeEV'].astype(np.float32)
    model['expEV'] = model['expEV'].astype(np.float32)
    model['expPC'] = model['expPC'].astype(np.float32)

    # matlab start with 1. change to 0 in python.
    model['tri'] = model['tri'].T.copy(order='C').astype(np.int32) - 1
    model['tri_mouth'] = model['tri_mouth'].T.copy(order='C').astype(
        np.int32) - 1
    # kpt ind
    model['kpt_ind'] = (np.squeeze(model['kpt_ind']) - 1).astype(np.int32)

    return model

In [4]:
head_model = load_BFM('/aidata/anders/objects/3D-head/3DDFA/BFM/BFM.mat')


In [8]:
kpt_ind = head_model['kpt_ind']
X_ind_all = np.stack(
    [kpt_ind * 3, kpt_ind * 3 + 1, kpt_ind * 3 + 2])
X_ind_all = tf.concat([
    X_ind_all[:, :17], X_ind_all[:, 17:27], X_ind_all[:, 36:48],
    X_ind_all[:, 27:36], X_ind_all[:, 48:68]
],
                      axis=-1)
valid_ind = np.reshape(np.transpose(X_ind_all), (-1))

shapeMU = head_model['shapeMU']
shapePC = head_model['shapePC'][:, :50] 
expPC = head_model['expPC'][:, :29]
sp = np.zeros(shape=(50, 1))
ep = np.zeros(shape=(29, 1))
X = shapeMU + shapePC.dot(sp) + expPC.dot(ep)
print(X.shape)

(159645, 1)


In [ ]:
#TODO: if I get the x rotated vertices
